## Comment the purpose of below cell

the findspark library is installed.
findspark is initiated

pyspark library is installed
SparkSession class from the pyspark.sql module is installed

In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession

## SparkSession for a spark application called csv2parquet is initiated

In [2]:
#Logger.getLogger("org").setLevel(Level.ERROR)
spark = SparkSession \
    .builder \
    .appName("csv2parquet") \
    .master("local[*]") \
    .config("spark.sql.warehouse.dir", "file:///tmp") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/30 03:17:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## configures spark to only display ERROR messages

In [3]:
sc=spark.sparkContext

In [4]:
sc.setLogLevel("ERROR")

## The code reads a csv file from /notebooks/ticker_symbol.csv into a data frame, ds.
## created a new df with colum headers to named as ["Ticker", "Name", "Exchange", "CategoryName", "CategoryNumber", "_c5"

In [11]:
ds = spark.read.format("csv").option("header", "false").option("quote", "\"").load("/notebooks/ticker_symbol.csv")

In [12]:
ds.show(truncate=False)

+------+-------------------------------+--------+---------------------------+--------------+----+
|_c0   |_c1                            |_c2     |_c3                        |_c4           |_c5 |
+------+-------------------------------+--------+---------------------------+--------------+----+
|Ticker|Name                           |Exchange|CategoryName               |CategoryNumber|NULL|
|AUB.AX|Austbrokers Holdings Limited   |ASX     |Accident & Health Insurance|431           |NULL|
|GLRE  |Greenlight Capital Re, Ltd.    |NMS     |Accident & Health Insurance|431           |NULL|
|SFG   |StanCorp Financial Group Inc.  |NYQ     |Accident & Health Insurance|431           |NULL|
|AMIC  |American Independence Corp.    |NGM     |Accident & Health Insurance|431           |NULL|
|GTS   |Triple-S Management Corporation|NYQ     |Accident & Health Insurance|431           |NULL|
|LRE.L |Lancashire Holdings Limited    |LSE     |Accident & Health Insurance|431           |NULL|
|EIG   |Employers Ho

In [13]:
heading = ["Ticker","Name","Exchange","CategoryName","CategoryNumber","_c5"]

In [14]:
df = ds.toDF(*heading)

## displays the df into a tabular format without truncating the data in the columns

In [15]:
df.show(truncate=False)

+------+-------------------------------+--------+---------------------------+--------------+----+
|Ticker|Name                           |Exchange|CategoryName               |CategoryNumber|_c5 |
+------+-------------------------------+--------+---------------------------+--------------+----+
|Ticker|Name                           |Exchange|CategoryName               |CategoryNumber|NULL|
|AUB.AX|Austbrokers Holdings Limited   |ASX     |Accident & Health Insurance|431           |NULL|
|GLRE  |Greenlight Capital Re, Ltd.    |NMS     |Accident & Health Insurance|431           |NULL|
|SFG   |StanCorp Financial Group Inc.  |NYQ     |Accident & Health Insurance|431           |NULL|
|AMIC  |American Independence Corp.    |NGM     |Accident & Health Insurance|431           |NULL|
|GTS   |Triple-S Management Corporation|NYQ     |Accident & Health Insurance|431           |NULL|
|LRE.L |Lancashire Holdings Limited    |LSE     |Accident & Health Insurance|431           |NULL|
|EIG   |Employers Ho

## A new dataframe is created called, df_with_datatype and it shows the first 3 rows

In [16]:
df_with_datatype=df.selectExpr("Ticker",\
                  "Name", \
                  "Exchange",\
                  "CategoryName",\
                  "cast(CategoryNumber as int) CategoryNumber")

In [17]:
df_with_datatype.show(3, False)

+------+----------------------------+--------+---------------------------+--------------+
|Ticker|Name                        |Exchange|CategoryName               |CategoryNumber|
+------+----------------------------+--------+---------------------------+--------------+
|Ticker|Name                        |Exchange|CategoryName               |NULL          |
|AUB.AX|Austbrokers Holdings Limited|ASX     |Accident & Health Insurance|431           |
|GLRE  |Greenlight Capital Re, Ltd. |NMS     |Accident & Health Insurance|431           |
+------+----------------------------+--------+---------------------------+--------------+
only showing top 3 rows



## The code saves the DataFrame df_with_datatype to a file in Parquet format at the /notebooks/ticker_symbol.parquet


In [18]:
#Save the DataFrame to Parquet format, overwrite if existing.
#Parquet is Columnar, good for Analytics query.

df_with_datatype.write.mode("Overwrite").parquet("/notebooks/ticker_symbol.parquet")


## The code reads a Parquet file from /notebooks/ticker_symbol.parquet into a dataframe read_parquet_df
## Displays the first 3 rows without truncation
## renames the colums to ("Ticker","Name","Exchange","CategoryName","CategoryNumber") into a new dataframe TickerSymbol

In [19]:
#Read the Parquet data back and run SQL query on it

read_parquet_df = spark.read.parquet("/notebooks/ticker_symbol.parquet")

read_parquet_df.show(3, False)

+------+----------------------------+--------+---------------------------+--------------+
|Ticker|Name                        |Exchange|CategoryName               |CategoryNumber|
+------+----------------------------+--------+---------------------------+--------------+
|Ticker|Name                        |Exchange|CategoryName               |NULL          |
|AUB.AX|Austbrokers Holdings Limited|ASX     |Accident & Health Insurance|431           |
|GLRE  |Greenlight Capital Re, Ltd. |NMS     |Accident & Health Insurance|431           |
+------+----------------------------+--------+---------------------------+--------------+
only showing top 3 rows



In [20]:
TickerSymbol = read_parquet_df.toDF("Ticker","Name","Exchange","CategoryName","CategoryNumber")
TickerSymbol.printSchema()


root
 |-- Ticker: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Exchange: string (nullable = true)
 |-- CategoryName: string (nullable = true)
 |-- CategoryNumber: integer (nullable = true)



## the code makes the TickerSymbol data available for SQL queries
## finds and shows rows for ('IBM','MSFT','HPQ','GE') from the data. 

In [21]:
TickerSymbol.createOrReplaceTempView("TickerSymbol")
spark.sql("SELECT * from TickerSymbol where Ticker in ('IBM','MSFT','HPQ','GE')").show(20,False)


+------+-------------------------------------------+--------+-------------------------------+--------------+
|Ticker|Name                                       |Exchange|CategoryName                   |CategoryNumber|
+------+-------------------------------------------+--------+-------------------------------+--------------+
|MSFT  |Microsoft Corporation                      |NMS     |Business Software & Services   |826           |
|HPQ   |Hewlett-Packard Company                    |NYQ     |Diversified Computer Systems   |810           |
|GE    |General Electric Company                   |NYQ     |Diversified Machinery          |622           |
|IBM   |International Business Machines Corporation|NYQ     |Information Technology Services|824           |
+------+-------------------------------------------+--------+-------------------------------+--------------+

